**3. Use the Intraday 1 Min data from parquet format, calculate Gain, GainP for every hourly window for each stock. The possible column output should be,
“SYMBOL”, “DATEFROM”, “DATETO”, “GAINP” where are DATEFROM and DATETO are timestamp/date columns example, DATEFROM from Feb 01, 2022 10:00 AM to Feb 01, 2022 11:00**

In [ ]:
dailyDf = spark.read.format("parquet")\
                .option("header", True)\
                .option("inferSchema", True)\
                .option("timestampFormat", "dd-MMM-yyyy")\
                .load("hdfs://localhost:9000/intraday/Year=2022/Month=02/Day=10")

dailyDf.printSchema()
dailyDf.show(5)

In [ ]:
dailyDf.groupBy("Symbol", F.window(dailyDf.DateTime, "60 minutes", "60 minutes"))

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag

windowSpec = Window.partitionBy('Symbol').orderBy('DateTime')

dailyDf = dailyDf.withColumn('DATEFROM', lag('DateTime').over(windowSpec))\
                .withColumnRenamed("DateTime", "DATETO")\
                .withColumn('GAIN', col('Close') -col('Open'))\
                .withColumn('GAINP', (col('GAIN') / col('Open')) *100)


dailyDf.printSch